# Lifetime spending with a risky asset

Trying to generalise the previous notebook, but with a risky asset.

We want to choose a consumption fraction $c_t$ and an investment fraction $k_t$ such that we maximize

$$\sum_{t=1}^T \frac{E(U(c_t W_t))}{(1+r_{tp})^t}$$

where $W_t = W_{t-1}(1-c_{t-1})(1+R_p)$ and $R_p = k_t R_r + (1-k_t)r_f$

where $r_f$ is the risk-free rate (after tax and inflation). $R_r$ is the return on a risky asset with $E(R_r) = \mu$ and $V(R_r) = \sigma^2$.

From previous work

$$E(U(W)) \approx U(E(W)) + \frac{U''(E(W))}{2} V(W)$$

$$U''(W) = -\gamma W^{-\gamma-1}$$

Therefore

$$E(U(c_t W_t)) \approx U(E(c_t W_t)) + \frac{U''(E(c_t W_t))}{2} V(c_t W_t)$$


## Compute mean and variance of $W_t$

$$E(W_t) = W_{t-1}(1-c_t)(1+(1-k_t)r_f) + W_{t-1}(1-c_t)k_t \mu$$

$$V(W_t) = W_{t-1}^2(1-c_t)^2k_t^2 \sigma^2$$

So then

$$E(U(c_t W_t)) \approx U(E(c_t W_t)) + \frac{U''(E(c_t W_t))}{2} V(c_t W_t)$$

$$E(c_t W_t) = c_tW_{t-1}(1-c_t)[1+(1-k_t)r_f + k_t \mu]$$

$$V(c_t W_t) = c_t^2 W_{t-1}^2(1-c_t)^2k_t^2 \sigma^2$$

In [1]:
import numpy as np 
import polars as pl 
from polars import col 
from tqdm.autonotebook import tqdm
from findec import crra_utility

from IPython.display import display

/tmp/ipykernel_10164/1453797730.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def crra_utility_second_derivative(w: np.ndarray | float, *, gamma: float):
    return -gamma * w ** -(gamma + 1)

In [3]:
initial_wealth = 1e6
GAMMA = 2.0
risk_adjusted_returns_per_year = 3e-2
future_utility_discount_rate_per_year = 1e-2
time_horizon = 20
mean_return_risky_asset = 11e-2
standard_deviation_risky_asset = 15e-2

In [ ]:
def solve_consumption(
    *,
    W0: float,
    r_f: float,
    r_tp: float,
    gamma: float,
    mu: float,
    sigma: float,
    T: int,
    n_grid: int,
    c_grid_size: int,
    k_grid_size: int,
):
    # 1. Build wealth grid
    max_growth_rate = max(r_f, mu + 1.5 * sigma)
    W_max = W0 * (1 + max_growth_rate) ** T * 2.0  # factor of 2 is for some margin
    W_grid = np.linspace(1e-3, W_max, n_grid)

    # 2. Arrays for value and policy 

    # V[t, i] holds the maximum discounted utiltiy achievable from time t onward if your current wealth is W_grid[i]
    V = np.zeros((T+2, n_grid))

    # P[t, i, j] holds the optimal (consumption fraction, risky asset fraction) that achieves that maximum
    P = np.zeros((T+2, n_grid, n_grid))

    # 3. Terminal condition: V[T+1] = 0
    # Already zero by default. No bequest

    # 4. Precompute discount factors
    discount_factors = [(1 + r_tp) ** t for t in range(T + 2)]

    # 5. Discretize consumption and risky asset fraction 
    c_candidates = np.linspace(0, 1, c_grid_size)
    k_candidates = np.linspace(0, 1, k_grid_size)

    # 6. Backwards induction
    for t in tqdm(reversed(range(1, T+1)), total=T):
        df = discount_factors[t]
        for i, W_now in W_grid:
            best_value = -np.inf 
            best_c = 0

            # Evaluate every candidate consumption value and risky asset allocation
            for c_frac in c_candidates:
                for k_frac in k_candidates:
                    